In [1]:
import tensorflow as tf
import numpy as np

**`Regularization`**

In [ ]:
tf.contrib.layers.l1_regularizer
tf.contrib.layers.l2_regularizer
tf.contrib.layers.l1_l2_regularizer
tf.contrib.layers.sum_regularizer
tf.contrib.layers.apply_regularization

In [ ]:
tf.contrib.layers.sum_regularizer(regularizer_list, scope=None)
返回一个可以执行多种(个)正则化的函数.意思是,创建一个正则化方法,这个方法是多个正则化方法的混合体.

In [ ]:
tf.contrib.layers.apply_regularization(regularizer, weights_list=None)
regularizer:就是我们上一步创建的正则化方法
weights_list: 想要执行正则化方法的参数列表,如果为None的话,就取GraphKeys.WEIGHTS中的weights

函数返回一个标量Tensor,同时,这个标量Tensor也会保存到GraphKeys.REGULARIZATION_LOSSES中.这个Tensor保存了计算正则项损失的方法.